# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd 
import xlsxwriter 
import math 
import requests
import yfinance as yf 
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [21]:
#grabbing list of S&P 500 Companies 
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]

symbols_list = (df['Symbol'])

stocks = symbols_list.to_frame()










    Symbol
0      MMM
1      AOS
2      ABT
3     ABBV
4     ABMD
..     ...
500    YUM
501   ZBRA
502    ZBH
503   ZION
504    ZTS

[505 rows x 1 columns]


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [11]:
aapl = yf.Ticker('AAPL')




<class 'dict'>


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [131]:
price = data['latestPrice']
pe_ratio = data['peRatio']


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [30]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))

symbol_strings = []




for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    

my_columns = ['Symbol', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AAP,AES,AFL,A,AIG,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AMD,AEE,AAL,AEP,AXP,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,APA,AAPL,AMAT,APTV,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CERN


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [36]:

simple_list = []

for symbol_string in symbol_strings[:1]: 
    print(len(symbol_string))
    
    for symbol in symbol_string.split(','):
        ticker = yf.Ticker(symbol)
        print(ticker.info)
        break



# final_dataframe = pd.DataFrame(simple_list, columns=my_columns)






        

426
{'zip': '55144-1000', 'sector': 'Industrials', 'fullTimeEmployees': 95000, 'longBusinessSummary': '3M Company develops, manufactures, and markets various products worldwide. It operates through four business segments: Safety and Industrial, Transportation and Electronics, Health Care, and Consumer. The Safety and Industrial segment offers personal safety products, industrial adhesives and tapes, abrasives, closure and masking systems, electrical markets, automotive aftermarket, and roofing granules to industrial, electrical, and safety markets. The Transportation and Electronics provides electronics, such as display materials and systems, electronic materials solutions; automotive and aerospace, and commercial solutions; advanced materials; and transportation safety products to transportation and electronic original equipment manufacturer customers. The Health Care segment offers medical and surgical supplies, skin health and infection prevention products, oral care, separation and

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [134]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]

final_dataframe.reset_index(inplace = True, drop = True)



## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [135]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [136]:
portfolio_input()


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [137]:
position_size = float(portfolio_size)/len(final_dataframe.index)
# position_size
# final_dataframe
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])




## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [138]:
# my_columns = ['Symbol', 'Price', 'Price-to-Earnings Ratio','Price-to-Book Ratio', 'Price-to-Sales Ratio' ,'Number of Shares to Buy']
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

pe_ratio = data[symbol]['quote']['peRatio']
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']

ev_to_ebitda = enterprise_value/ebitda
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit



Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [139]:
rv_columns = [
    'Symbol',
    'Price',
    'Number of Shares to Buy',
    'PE Ratio',
    'PE Percentile',
    'PB Ratio',
    'PB Percentile',
    'PS Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'

]


In [140]:
simple_list = []

for symbol_string in symbol_strings: 
    batch_api_call_url = batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'WLTW':
            pass
        else:
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']

            try:
                ev_to_ebitda = enterprise_value/ebitda
            
            except TypeError:
                ev_to_ebitda = np.NaN

            
            
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            
            gross_profit = data[symbol]['advanced-stats']['grossProfit']

            simple_list.append([symbol, data[symbol]['quote']['latestPrice'], 'N/A', data[symbol]['quote']['peRatio'], 'N/A', data[symbol]['advanced-stats']['priceToBook'], 'N/A', data[symbol]['advanced-stats']['priceToSales'], 'N/A', ev_to_ebitda, 'N/A',ev_to_gross_profit, 'N/A', 'N/A'])


rv_dataframe = pd.DataFrame(simple_list, columns = rv_columns)





## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [141]:
rv_dataframe

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,150.46,N/A,18.69,N/A,4.15,N/A,3.530,N/A,13.896669,N/A,9.503612,N/A,N/A
1,AAL,19.48,N/A,-3.86,N/A,-1.72,N/A,0.505,N/A,-7.211433,N/A,9.503612,N/A,N/A
2,AAP,236.98,N/A,24.58,N/A,4.72,N/A,1.370,N/A,13.460067,N/A,9.503612,N/A,N/A
3,AAPL,178.23,N/A,16.22,N/A,22.36,N/A,3.980,N/A,12.713962,N/A,9.503612,N/A,N/A
4,ABBV,140.39,N/A,33.69,N/A,18.16,N/A,4.530,N/A,11.338550,N/A,9.503612,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,132.61,N/A,26.01,N/A,-5.03,N/A,6.360,N/A,21.746710,N/A,9.503612,N/A,N/A
500,ZBH,132.19,N/A,33.88,N/A,2.17,N/A,3.510,N/A,18.349037,N/A,9.503612,N/A,N/A
501,ZBRA,552.30,N/A,34.66,N/A,10.66,N/A,5.270,N/A,26.102508,N/A,9.503612,N/A,N/A
502,ZION,71.60,N/A,6.80,N/A,1.53,N/A,3.140,N/A,5.218946,N/A,9.503612,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [142]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,188.300,N/A,61.66,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
40,AON,289.400,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
71,BRK.B,324.700,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
118,CTL,11.000,N/A,9.99,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
136,DISCK,28.570,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
165,ETFC,51.450,N/A,14.75,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
186,FLIR,59.660,N/A,32.64,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
190,FOX,36.160,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
193,FRT,135.900,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A
204,GOOG,2833.530,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,9.503612,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [143]:

exclude = []

print(rv_columns[3:12:2])


# print(value)
for column in rv_columns[3:12:2]:

    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace= True)



['PE Ratio', 'PB Ratio', 'PS Ratio', 'EV/EBITDA', 'EV/GP']


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [145]:
metrics = {}

for column in rv_dataframe.columns[3:12:2]:
    column1 = rv_dataframe.columns.get_loc(column)
    metrics.update({column : rv_dataframe.columns[column1 + 1] })

for metric in metrics: 
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])



    Symbol   Price Number of Shares to Buy  PE Ratio PE Percentile  PB Ratio  \
0        A  150.46                     N/A     18.69     36.507937      4.15   
1      AAL   19.48                     N/A     -3.86      5.357143     -1.72   
2      AAP  236.98                     N/A     24.58     50.793651      4.72   
3     AAPL  178.23                     N/A     16.22     29.960317     22.36   
4     ABBV  140.39                     N/A     33.69     67.063492     18.16   
..     ...     ...                     ...       ...           ...       ...   
499    YUM  132.61                     N/A     26.01     54.761905     -5.03   
500    ZBH  132.19                     N/A     33.88      67.65873      2.17   
501   ZBRA  552.30                     N/A     34.66     69.444444     10.66   
502   ZION   71.60                     N/A      6.80       9.52381      1.53   
503    ZTS  218.10                     N/A     51.65     83.829365     22.20   

    PB Percentile  PS Ratio PS Percenti

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [153]:
from statistics import mean



for row in rv_dataframe.index:
    pe = rv_dataframe.loc[row, 'PE Percentile']
    pb = rv_dataframe.loc[row, 'PB Percentile']
    ps = rv_dataframe.loc[row, 'PS Percentile']
    ev_ebitda = rv_dataframe.loc[row, 'EV/EBITDA Percentile']
    ev_gp = rv_dataframe.loc[row, 'EV/GP Percentile'] 
    rv_dataframe.loc[row, 'RV Score'] = mean([pe, pb, ps, ev_ebitda, ev_gp])



## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [164]:
best_value_stocks = rv_dataframe.sort_values(by = 'RV Score', ascending = False)
best_value_stocks = best_value_stocks[:50]
best_value_stocks.reset_index(drop = True, inplace = True )

best_value_stocks

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,599.54,N/A,554.83,99.603175,32.50,96.031746,20.65,97.81746,164.852600,99.801587,9.503612,50.099206,88.670635
1,PAYC,379.80,N/A,129.49,96.825397,26.58,94.84127,22.63,98.809524,77.694442,98.015873,9.503612,50.099206,87.718254
2,NVDA,291.37,N/A,86.36,95.238095,30.19,95.634921,28.31,99.404762,68.196968,97.619048,9.503612,50.099206,87.599206
3,FTNT,333.49,N/A,99.96,96.031746,47.80,97.222222,17.34,96.626984,76.149106,97.81746,9.503612,50.099206,87.559524
4,DXCM,489.28,N/A,93.36,95.634921,21.06,93.253968,19.59,97.222222,95.981026,99.404762,9.503612,50.099206,87.123016
5,IDXX,566.66,N/A,65.36,91.666667,64.85,98.412698,15.56,95.238095,46.328714,95.634921,9.503612,50.099206,86.210317
6,CDNS,169.71,N/A,69.37,92.65873,18.29,92.460317,15.49,95.039683,55.806613,97.222222,9.503612,50.099206,85.496032
7,BKNG,2562.69,N/A,266.88,98.611111,18.20,92.261905,11.12,89.880952,48.754404,96.031746,9.503612,50.099206,85.376984
8,INTU,603.50,N/A,78.44,94.444444,16.09,90.674603,15.32,94.444444,54.892326,96.825397,9.503612,50.099206,85.297619
9,ISRG,340.51,N/A,73.84,93.650794,10.36,83.134921,21.54,98.611111,52.919474,96.626984,9.503612,50.099206,84.424603


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [160]:
portfolio_input()


In [166]:
for row in best_value_stocks.index:
    best_value_stocks.loc[row, 'Number of Shares to Buy'] = math.floor(float(portfolio_size)/best_value_stocks.loc[row, 'Price'])

best_value_stocks

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,599.54,16679,554.83,99.603175,32.50,96.031746,20.65,97.81746,164.852600,99.801587,9.503612,50.099206,88.670635
1,PAYC,379.80,26329,129.49,96.825397,26.58,94.84127,22.63,98.809524,77.694442,98.015873,9.503612,50.099206,87.718254
2,NVDA,291.37,34320,86.36,95.238095,30.19,95.634921,28.31,99.404762,68.196968,97.619048,9.503612,50.099206,87.599206
3,FTNT,333.49,29985,99.96,96.031746,47.80,97.222222,17.34,96.626984,76.149106,97.81746,9.503612,50.099206,87.559524
4,DXCM,489.28,20438,93.36,95.634921,21.06,93.253968,19.59,97.222222,95.981026,99.404762,9.503612,50.099206,87.123016
5,IDXX,566.66,17647,65.36,91.666667,64.85,98.412698,15.56,95.238095,46.328714,95.634921,9.503612,50.099206,86.210317
6,CDNS,169.71,58924,69.37,92.65873,18.29,92.460317,15.49,95.039683,55.806613,97.222222,9.503612,50.099206,85.496032
7,BKNG,2562.69,3902,266.88,98.611111,18.20,92.261905,11.12,89.880952,48.754404,96.031746,9.503612,50.099206,85.376984
8,INTU,603.50,16570,78.44,94.444444,16.09,90.674603,15.32,94.444444,54.892326,96.825397,9.503612,50.099206,85.297619
9,ISRG,340.51,29367,73.84,93.650794,10.36,83.134921,21.54,98.611111,52.919474,96.626984,9.503612,50.099206,84.424603


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: